In [1]:
!pip install convert_numbers
!pip install hazm
!pip install selenium
!pip install elasticsearch

In [1]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import NoSuchElementException, TimeoutException, StaleElementReferenceException
import time
import pandas as pd
from elasticsearch import Elasticsearch, helpers
import re
from bs4 import BeautifulSoup
import convert_numbers
import warnings

import nltk
import numpy as np
import pandas as pd
from sklearn.metrics.pairwise import cosine_similarity


In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
def scroll_and_load(driver, pause_time=3, scroll_step=400):
    last_height = driver.execute_script("return document.body.scrollHeight")
    for i in range(0, last_height, scroll_step):
            driver.execute_script(f"window.scrollTo(0, {i});")
            time.sleep(pause_time)  # Wait to load the comments/buttons



In [ ]:
driver = webdriver.Chrome()
url = 'https://www.digikala.com/product/dkp-856812/%D9%BE%D9%86%DB%8C%D8%B1-%D8%AE%D8%A7%D9%85%D9%87-%D8%A7%DB%8C-%D9%88%DB%8C%D9%84%DB%8C-%DA%A9%D8%A7%D9%84%D9%87-350-%DA%AF%D8%B1%D9%85/'
driver.get(url)
time.sleep(8)
driver.maximize_window()
time.sleep(3)
scroll_and_load(driver, pause_time=3, scroll_step=400)
expand_comments_button = WebDriverWait(driver, 10).until(
    EC.element_to_be_clickable((By.XPATH, '//button[contains(@class, "styles_btn__Q4MvL") and contains(@class, "text-secondary-500")]'))
)

# Click the button to expand the comments section
driver.execute_script("arguments[0].click();", expand_comments_button)

In [ ]:
def get_comments(page_source,data):
    # Use BeautifulSoup to parse the page source
    soup = BeautifulSoup(page_source, 'html.parser')

    # Find all comment articles
    comment_articles = soup.find_all('article', class_='py-3 lg:mt-0 br-list-vertical-no-padding-200')

    # Extract comments and labels
    comments_data = []
    for article in comment_articles:
        comment_text = article.find('p', class_='text-body-1 text-neutral-900 mb-1 pt-3 break-words')
        if comment_text :
            comment_text = comment_text.get_text(strip=True)
            comments_data.append({'comment': comment_text})
    #print(comments_data)
    data.extend(comments_data)

In [ ]:
full_data=[]

In [ ]:
get_comments(driver.page_source,full_data)

In [ ]:
for i in range(2,100,1):
    time.sleep(5)  # Give it a moment to scroll

    next_page_button = WebDriverWait(driver, 10).until(
        EC.element_to_be_clickable((By.XPATH, f'//span[contains(@class, "styles_PageNumberButton__yPGpm") and span[text()="{convert_numbers.english_to_persian(str(i))}"]]'))
    )
    driver.execute_script("arguments[0].scrollIntoView(true);", next_page_button)
    time.sleep(3)  # Give it a moment to scroll
    driver.execute_script("arguments[0].click();", next_page_button)
    time.sleep(3)  # Give it a moment to scroll

    get_comments(driver.page_source,full_data)

all_data = pd.DataFrame(full_data)


In [ ]:
all_data

,comment
0,بنظرم در بین پنیر خامه‌ای ها از همه بهتره و بر...
1,خوب بود
2,خوش طعم و با کیفیت
3,عالی
4,این طعم رو دوست داریم
...,...
415,خوشمزه و عالیه
416,خوبه خوشمزس. دوسش داشتیم.
417,لطیف و خوش طعم
418,ممنون دیجی کالا


In [5]:
username = "elastic"
password = "ctXPvcvEnf1NN4eyuT4E"
# Connect to Elasticsearch
es = Elasticsearch([{'host': 'localhost', 'port': 9200, 'scheme': 'https'}],ca_certs=False, verify_certs=False,http_auth=(username,password))
# Convert DataFrame to list of dictionaries
data_dict = all_data.to_dict(orient='records')
# Create the Elasticsearch index (optional, only if you need to create it explicitly)
index_name = 'digikala_comments'
if not es.indices.exists(index=index_name):
    es.indices.create(index=index_name)

# Prepare data for bulk upload
actions = [
    {
        "_index": index_name,
        "_source": data
    }
    for data in data_dict
]

# Bulk upload data to Elasticsearch
helpers.bulk(es, actions)

C:\Users\ZETTA\anaconda3\lib\site-packages\elasticsearch\_sync\client\__init__.py:399: SecurityWarning: Connecting to 'https://localhost:9200' using TLS with verify_certs=False is insecure
  _transport = transport_class(
C:\Users\ZETTA\AppData\Local\Temp\ipykernel_28044\3894791709.py:4: DeprecationWarning: The 'http_auth' parameter is deprecated. Use 'basic_auth' or 'bearer_auth' parameters instead
  es = Elasticsearch([{'host': 'localhost', 'port': 9200, 'scheme': 'https'}],ca_certs=False, verify_certs=False,http_auth=(username,password))
C:\Users\ZETTA\anaconda3\lib\site-packages\urllib3\connectionpool.py:1045: InsecureRequestWarning: Unverified HTTPS request is being made to host 'localhost'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(
C:\Users\ZETTA\anaconda3\lib\site-packages\urllib3\connectionpool.py:1045: InsecureRequestWarning: Unverified HTTPS request is being made to host 

(420, [])

In [6]:
# first download the postagger model from https://drive.google.com/file/d/1Q3JK4NVUC2t5QT63aDiVrCRBV225E_B3
from hazm import *
normalizer = Normalizer()
word_tokenizer = WordTokenizer()
stemmer = Stemmer()
lemmatizer = Lemmatizer()
tagger = POSTagger(model ='/content/drive/MyDrive/pos_tagger.model')

In [7]:
def process_text(text):
    normalized_text = normalizer.normalize(text)
    words = word_tokenizer.tokenize(normalized_text)
    stems = [stemmer.stem(word) for word in words]
    lemmas = [lemmatizer.lemmatize(word) for word in words]
    pos_tags = tagger.tag(words)
    return {
        'normalized': normalized_text,
        'words': words,
        'stems': stems,
        'lemmas': lemmas,
        'pos_tags': pos_tags
    }

In [8]:
# Process all comments
all_data['processed'] = all_data['comment'].apply(process_text)

In [9]:
# Extract process columns
all_data['normalized'] = all_data['processed'].apply(lambda x: x['normalized'])
all_data['words'] = all_data['processed'].apply(lambda x: x['words'])
all_data['stems'] = all_data['processed'].apply(lambda x: x['stems'])
all_data['lemmas'] = all_data['processed'].apply(lambda x: x['lemmas'])
all_data['pos_tags'] = all_data['processed'].apply(lambda x: x['pos_tags'])
all_data

,comment,processed,normalized,words,stems,lemmas,pos_tags
0,بنظرم در بین پنیر خامه‌ای ها از همه بهتره و بر...,{'normalized': 'بنظرم در بین پنیر خامه‌ای‌ها ا...,بنظرم در بین پنیر خامه‌ای‌ها از همه بهتره و بر...,"[بنظرم, در, بین, پنیر, خامه‌ای‌ها, از, همه, به...","[بنظر, در, بین, پنیر, خامه‌ای, از, همه, بهتره,...","[بنظر, در, بین, پنیر, خامه‌ای‌ها, از, همه, بهت...","[(بنظرم, VERB), (در, ADP), (بین, NOUN,EZ), (پن..."
1,خوب بود,"{'normalized': 'خوب بود', 'words': ['خوب', 'بو...",خوب بود,"[خوب, بود]","[خوب, بود]","[خوب, بود#است]","[(خوب, ADJ), (بود, VERB)]"
2,خوش طعم و با کیفیت,"{'normalized': 'خوش‌طعم و با کیفیت', 'words': ...",خوش‌طعم و با کیفیت,"[خوش‌طعم, و, با, کیفیت]","[خوش‌طع, و, با, کیف]","[خوش‌طعم, و, با, کیفیت]","[(خوش‌طعم, NOUN), (و, CCONJ), (با, ADP), (کیفی..."
3,عالی,"{'normalized': 'عالی', 'words': ['عالی'], 'ste...",عالی,[عالی],[عال],[عالی],"[(عالی, ADJ)]"
4,این طعم رو دوست داریم,"{'normalized': 'این طعم رو دوست داریم', 'words...",این طعم رو دوست داریم,"[این, طعم, رو, دوست, داریم]","[این, طع, رو, دوس, دار]","[این, طعم, رو, دوست, داشت#دار]","[(این, DET), (طعم, NOUN), (رو, ADP), (دوست, NO..."
...,...,...,...,...,...,...,...
415,خوشمزه و عالیه,"{'normalized': 'خوشمزه و عالیه', 'words': ['خو...",خوشمزه و عالیه,"[خوشمزه, و, عالیه]","[خوشمزه, و, عالیه]","[خوشمزه, و, عالیه]","[(خوشمزه, ADJ), (و, CCONJ), (عالیه, ADJ)]"
416,خوبه خوشمزس. دوسش داشتیم.,"{'normalized': 'خوبه خوشمزس. دوسش داشتیم.', 'w...",خوبه خوشمزس. دوسش داشتیم.,"[خوبه, خوشمزس, ., دوسش, داشتیم, .]","[خوبه, خوشمزس, ., دوس, داشت, .]","[خوبه, خوشمزس, ., دوس, داشت#دار, .]","[(خوبه, VERB), (خوشمزس, NOUN), (., PUNCT), (دو..."
417,لطیف و خوش طعم,"{'normalized': 'لطیف و خوش‌طعم', 'words': ['لط...",لطیف و خوش‌طعم,"[لطیف, و, خوش‌طعم]","[لطیف, و, خوش‌طع]","[لطیف, و, خوش‌طعم]","[(لطیف, ADJ), (و, CCONJ), (خوش‌طعم, ADJ)]"
418,ممنون دیجی کالا,"{'normalized': 'ممنون دیجی کالا', 'words': ['م...",ممنون دیجی کالا,"[ممنون, دیجی, کالا]","[ممنون, دیج, کالا]","[ممنون, دیج, کالا]","[(ممنون, NOUN,EZ), (دیجی, ADJ,EZ), (کالا, NOUN)]"


In [6]:
indexes = {
    'normalized': 'comments_normalized',
    'words': 'comments_words',
    'stems': 'comments_stems',
    'lemmas': 'comments_lemmas',
    'pos_tags': 'comments_pos_tags'
}

# Create indexes
for index in indexes.values():
    if not es.indices.exists(index=index):
        es.indices.create(index=index)





C:\Users\ZETTA\anaconda3\lib\site-packages\urllib3\connectionpool.py:1045: InsecureRequestWarning: Unverified HTTPS request is being made to host 'localhost'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(
C:\Users\ZETTA\anaconda3\lib\site-packages\urllib3\connectionpool.py:1045: InsecureRequestWarning: Unverified HTTPS request is being made to host 'localhost'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(
C:\Users\ZETTA\anaconda3\lib\site-packages\urllib3\connectionpool.py:1045: InsecureRequestWarning: Unverified HTTPS request is being made to host 'localhost'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(
C:\Users\ZETTA\anaconda3\lib\site-packages\urllib3\connectionpool.py:1045: I

In [7]:
# Bulk upload data to Elasticsearch
def bulk_upload(index, data):
    actions = [
        {
            "_index": index,
            "_source": doc
        }
        for doc in data
    ]
    helpers.bulk(es, actions)

In [9]:
bulk_upload(indexes['normalized'], all_data[['comment', 'normalized']].to_dict(orient='records'))
bulk_upload(indexes['words'], all_data[['comment', 'words']].to_dict(orient='records'))
bulk_upload(indexes['stems'], all_data[['comment' ,'stems']].to_dict(orient='records'))
bulk_upload(indexes['lemmas'], all_data[['comment', 'lemmas']].to_dict(orient='records'))
bulk_upload(indexes['pos_tags'], all_data[['comment', 'pos_tags']].to_dict(orient='records'))

C:\Users\ZETTA\anaconda3\lib\site-packages\urllib3\connectionpool.py:1045: InsecureRequestWarning: Unverified HTTPS request is being made to host 'localhost'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(
C:\Users\ZETTA\anaconda3\lib\site-packages\urllib3\connectionpool.py:1045: InsecureRequestWarning: Unverified HTTPS request is being made to host 'localhost'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(
C:\Users\ZETTA\anaconda3\lib\site-packages\urllib3\connectionpool.py:1045: InsecureRequestWarning: Unverified HTTPS request is being made to host 'localhost'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(
C:\Users\ZETTA\anaconda3\lib\site-packages\urllib3\connectionpool.py:1045: I

In [56]:
grammers = [
    """
NP:
        {<NOUN,EZ>?<NOUN.*>}    # Noun(s) + Noun(optional)

""",
    """
NP:
        {<NOUN.*><ADJ.*>?}    # Noun(s) + Adjective(optional)

""",
]

def tokenize(text):
    normalizer = Normalizer()
    return [
        word_tokenize(sent)
        for sent in sent_tokenize(normalizer.normalize(text))
    ]


def posTagger(text, pos_model_path="/content/drive/MyDrive/pos_tagger.model", posTaggerModel=None):
    tokens = tokenize(text)
    tagger = POSTagger(pos_model_path) if posTaggerModel is None else posTaggerModel
    return tagger.tag_sents(tokens)


def extractGrammer(tagged_text, grammer):
    keyphrase_candidate = set()
    np_parser = nltk.RegexpParser(grammer)
    trees = np_parser.parse_sents(tagged_text)
    for tree in trees:
        for subtree in tree.subtrees(
            filter=lambda t: t.label() == "NP",
        ):  # For each nounphrase
            keyphrase_candidate.add(" ".join(word for word, tag in subtree.leaves()))
    keyphrase_candidate = {kp for kp in keyphrase_candidate if len(kp.split()) <= 5}
    keyphrase_candidate = list(keyphrase_candidate)
    return keyphrase_candidate


def extractCandidates(tagged_text, grammers=grammers):
    all_candidates = set()
    for grammer in grammers:
        all_candidates.update(extractGrammer(tagged_text, grammer))
    return np.array(list(all_candidates))


def text2vec(candidates, sent2vec_model_path="/content/drive/MyDrive/Colab Notebooks/sent2vec (1).zip (Unzipped Files)/sent2vec-naab.model", sent2vecModel=None,i=0):
    sent2vec_model = (
        SentEmbedding(sent2vec_model_path) if sent2vecModel is None else sent2vecModel
    )
    candidate_vector = [[sent2vec_model[candidate] for candidate in candidates]]
    text_vector = sent2vec_model[" ".join(candidates)]
    return candidate_vector, text_vector


def vectorSimilarity(candidates_vector, text_vector, norm=True):

    if len(candidates_vector[0]) == 0 :
        return np.array([]), np.array([])

    candidate_sim_text = cosine_similarity(
        candidates_vector[0],
        text_vector.reshape(1, -1),
    )
    candidate_sim_candidate = cosine_similarity(candidates_vector[0])

    if norm:
        candidates_sim_text_norm = candidate_sim_text / np.max(candidate_sim_text)
        candidates_sim_text_norm = 0.5 + (
            candidates_sim_text_norm - np.average(candidates_sim_text_norm)
        ) / np.std(candidates_sim_text_norm)
        np.fill_diagonal(candidate_sim_candidate, np.NaN)
        candidate_sim_candidate_norm = candidate_sim_candidate / np.nanmax(
            candidate_sim_candidate,
            axis=0,
        )
        candidate_sim_candidate_norm = 0.5 + (
            candidate_sim_candidate_norm
            - np.nanmean(candidate_sim_candidate_norm, axis=0)
        ) / np.nanstd(candidate_sim_candidate_norm, axis=0)

        return candidates_sim_text_norm, candidate_sim_candidate_norm

    return candidate_sim_text, candidate_sim_candidate




'''def extractKeyword(candidates, keyword_num=5, sent2vecModel=None):
    candidates_vector, text_vector = text2vec(candidates, sent2vecModel=sent2vecModel)
    candidate_sim_text_norm, candidate_sim_candidate_norm = vectorSimilarity(
        candidates_vector,
        text_vector,
    )
    return embedRankExtraction(
        candidates,
        candidate_sim_text_norm,
        candidate_sim_candidate_norm,
        keyword_num,
    )
'''
# Define the embedRankExtraction function
def embedRankExtraction(
    all_candidates,
    candidate_sim_text,
    candidate_sim_candidate,
    keyword_num=10,
    beta=0.8,
):
    if len(all_candidates)==0 or len(candidate_sim_text)==0 or len(candidate_sim_candidate)==0:
        return []

    if len(all_candidates) < keyword_num:
        warnings.warn(
            (
                f"total number of keyword candidates is {len(all_candidates)}, which is"
                " lower than your request keyword_num"
            ),
        )

    N = int(min(len(all_candidates), keyword_num))

    selected_candidates = []
    unselected_candidates = list(range(len(all_candidates)))
    best_candidate = np.argmax(candidate_sim_text)
    selected_candidates.append(best_candidate)
    unselected_candidates.remove(best_candidate)

    for _i in range(N - 1):
        selected_vec = np.array(selected_candidates)
        unselected_vec = np.array(unselected_candidates)

        unselected_candidate_sim_text = candidate_sim_text[unselected_vec, :]

        dist_between = candidate_sim_candidate[unselected_vec][:, selected_vec]

        if dist_between.ndim == 1:
            dist_between = dist_between[:, np.newaxis]

        best_candidate = np.argmax(
            beta * unselected_candidate_sim_text
            - (1 - beta) * np.max(dist_between, axis=1).reshape(-1, 1),
        )
        best_index = unselected_candidates[best_candidate]
        selected_candidates.append(best_index)
        unselected_candidates.remove(best_index)
    return all_candidates[selected_candidates].tolist()

In [13]:
#results = all_data['comment'].apply(lambda x: embedRank(x, keyword_num=5))

all_data['candidate_tags'] = all_data["comment"].apply(lambda x: extractCandidates(posTagger(x)))


#all_data['cosine_sim_text'] = results.apply(lambda x: x[1].flatten().tolist())
#all_data['cosine_sim_candidates'] = results.apply(lambda x: x[2].tolist())
#all_data['keywords'] = results.apply(lambda x: x[3])

In [14]:
all_data

,comment,processed,normalized,words,stems,lemmas,pos_tags,candidate_tags
0,بنظرم در بین پنیر خامه‌ای ها از همه بهتره و بر...,{'normalized': 'بنظرم در بین پنیر خامه‌ای‌ها ا...,بنظرم در بین پنیر خامه‌ای‌ها از همه بهتره و بر...,"[بنظرم, در, بین, پنیر, خامه‌ای‌ها, از, همه, به...","[بنظر, در, بین, پنیر, خامه‌ای, از, همه, بهتره,...","[بنظر, در, بین, پنیر, خامه‌ای‌ها, از, همه, بهت...","[(بنظرم, VERB), (در, ADP), (بین, NOUN,EZ), (پن...","[پنیر, کیفیت, بین, خامه‌ای‌ها, برندها, بین پنیر]"
1,خوب بود,"{'normalized': 'خوب بود', 'words': ['خوب', 'بو...",خوب بود,"[خوب, بود]","[خوب, بود]","[خوب, بود#است]","[(خوب, ADJ), (بود, VERB)]",[]
2,خوش طعم و با کیفیت,"{'normalized': 'خوش‌طعم و با کیفیت', 'words': ...",خوش‌طعم و با کیفیت,"[خوش‌طعم, و, با, کیفیت]","[خوش‌طع, و, با, کیف]","[خوش‌طعم, و, با, کیفیت]","[(خوش‌طعم, NOUN), (و, CCONJ), (با, ADP), (کیفی...","[خوش‌طعم, کیفیت]"
3,عالی,"{'normalized': 'عالی', 'words': ['عالی'], 'ste...",عالی,[عالی],[عال],[عالی],"[(عالی, ADJ)]",[]
4,این طعم رو دوست داریم,"{'normalized': 'این طعم رو دوست داریم', 'words...",این طعم رو دوست داریم,"[این, طعم, رو, دوست, داریم]","[این, طع, رو, دوس, دار]","[این, طعم, رو, دوست, داشت#دار]","[(این, DET), (طعم, NOUN), (رو, ADP), (دوست, NO...","[دوست, طعم]"
...,...,...,...,...,...,...,...,...
415,خوشمزه و عالیه,"{'normalized': 'خوشمزه و عالیه', 'words': ['خو...",خوشمزه و عالیه,"[خوشمزه, و, عالیه]","[خوشمزه, و, عالیه]","[خوشمزه, و, عالیه]","[(خوشمزه, ADJ), (و, CCONJ), (عالیه, ADJ)]",[]
416,خوبه خوشمزس. دوسش داشتیم.,"{'normalized': 'خوبه خوشمزس. دوسش داشتیم.', 'w...",خوبه خوشمزس. دوسش داشتیم.,"[خوبه, خوشمزس, ., دوسش, داشتیم, .]","[خوبه, خوشمزس, ., دوس, داشت, .]","[خوبه, خوشمزس, ., دوس, داشت#دار, .]","[(خوبه, VERB), (خوشمزس, NOUN), (., PUNCT), (دو...","[دوسش, خوشمزس]"
417,لطیف و خوش طعم,"{'normalized': 'لطیف و خوش‌طعم', 'words': ['لط...",لطیف و خوش‌طعم,"[لطیف, و, خوش‌طعم]","[لطیف, و, خوش‌طع]","[لطیف, و, خوش‌طعم]","[(لطیف, ADJ), (و, CCONJ), (خوش‌طعم, ADJ)]",[]
418,ممنون دیجی کالا,"{'normalized': 'ممنون دیجی کالا', 'words': ['م...",ممنون دیجی کالا,"[ممنون, دیجی, کالا]","[ممنون, دیج, کالا]","[ممنون, دیج, کالا]","[(ممنون, NOUN,EZ), (دیجی, ADJ,EZ), (کالا, NOUN)]","[ممنون دیجی, ممنون, کالا]"


In [19]:


# Function that returns a tuple of (candidate_vector, text_vector)
def text2vec_wrapper(candidate_tags):
    return text2vec(candidate_tags)

# Apply the function and create a temporary DataFrame
temp_df = all_data['candidate_tags'].apply(text2vec_wrapper)

# Assign the two columns from the temporary DataFrame to your original DataFrame
all_data['cand_vector'] = temp_df.apply(lambda x: x[0])
all_data['text_vector'] = temp_df.apply(lambda x: x[1])

#import ace_tools as tools; tools.display_dataframe_to_user(name="Processed Data", dataframe=all_data)


In [20]:
all_data

,comment,processed,normalized,words,stems,lemmas,pos_tags,candidate_tags,cand_vector,text_vector
0,بنظرم در بین پنیر خامه‌ای ها از همه بهتره و بر...,{'normalized': 'بنظرم در بین پنیر خامه‌ای‌ها ا...,بنظرم در بین پنیر خامه‌ای‌ها از همه بهتره و بر...,"[بنظرم, در, بین, پنیر, خامه‌ای‌ها, از, همه, به...","[بنظر, در, بین, پنیر, خامه‌ای, از, همه, بهتره,...","[بنظر, در, بین, پنیر, خامه‌ای‌ها, از, همه, بهت...","[(بنظرم, VERB), (در, ADP), (بین, NOUN,EZ), (پن...","[پنیر, کیفیت, بین, خامه‌ای‌ها, برندها, بین پنیر]","[[[-0.04970275, -0.051454887, 0.020210866, 0.0...","[0.09832931, 0.016934274, 0.19831714, 0.065371..."
1,خوب بود,"{'normalized': 'خوب بود', 'words': ['خوب', 'بو...",خوب بود,"[خوب, بود]","[خوب, بود]","[خوب, بود#است]","[(خوب, ADJ), (بود, VERB)]",[],[[]],"[0.0010530046, -0.00035910666, -0.00092692504,..."
2,خوش طعم و با کیفیت,"{'normalized': 'خوش‌طعم و با کیفیت', 'words': ...",خوش‌طعم و با کیفیت,"[خوش‌طعم, و, با, کیفیت]","[خوش‌طع, و, با, کیف]","[خوش‌طعم, و, با, کیفیت]","[(خوش‌طعم, NOUN), (و, CCONJ), (با, ADP), (کیفی...","[خوش‌طعم, کیفیت]","[[[-0.028009595, -0.018868402, 0.026083963, 0....","[0.030181762, -0.025148587, 0.032836657, 0.075..."
3,عالی,"{'normalized': 'عالی', 'words': ['عالی'], 'ste...",عالی,[عالی],[عال],[عالی],"[(عالی, ADJ)]",[],[[]],"[-0.0006072971, -0.0010746294, -0.00013960322,..."
4,این طعم رو دوست داریم,"{'normalized': 'این طعم رو دوست داریم', 'words...",این طعم رو دوست داریم,"[این, طعم, رو, دوست, داریم]","[این, طع, رو, دوس, دار]","[این, طعم, رو, دوست, داشت#دار]","[(این, DET), (طعم, NOUN), (رو, ADP), (دوست, NO...","[دوست, طعم]","[[[-0.055163667, -0.07642987, -0.017042464, 0....","[-0.054517377, -0.090128876, -0.0017683336, 0...."
...,...,...,...,...,...,...,...,...,...,...
415,خوشمزه و عالیه,"{'normalized': 'خوشمزه و عالیه', 'words': ['خو...",خوشمزه و عالیه,"[خوشمزه, و, عالیه]","[خوشمزه, و, عالیه]","[خوشمزه, و, عالیه]","[(خوشمزه, ADJ), (و, CCONJ), (عالیه, ADJ)]",[],[[]],"[0.0013152182, -0.00048353066, -0.0013701833, ..."
416,خوبه خوشمزس. دوسش داشتیم.,"{'normalized': 'خوبه خوشمزس. دوسش داشتیم.', 'w...",خوبه خوشمزس. دوسش داشتیم.,"[خوبه, خوشمزس, ., دوسش, داشتیم, .]","[خوبه, خوشمزس, ., دوس, داشت, .]","[خوبه, خوشمزس, ., دوس, داشت#دار, .]","[(خوبه, VERB), (خوشمزس, NOUN), (., PUNCT), (دو...","[دوسش, خوشمزس]","[[[0.0003717778, -0.037761122, -0.011944341, -...","[0.030795809, -0.0028205423, 0.01159153, -0.01..."
417,لطیف و خوش طعم,"{'normalized': 'لطیف و خوش‌طعم', 'words': ['لط...",لطیف و خوش‌طعم,"[لطیف, و, خوش‌طعم]","[لطیف, و, خوش‌طع]","[لطیف, و, خوش‌طعم]","[(لطیف, ADJ), (و, CCONJ), (خوش‌طعم, ADJ)]",[],[[]],"[0.001214448, -0.0012010353, 0.0016182135, -0...."
418,ممنون دیجی کالا,"{'normalized': 'ممنون دیجی کالا', 'words': ['م...",ممنون دیجی کالا,"[ممنون, دیجی, کالا]","[ممنون, دیج, کالا]","[ممنون, دیج, کالا]","[(ممنون, NOUN,EZ), (دیجی, ADJ,EZ), (کالا, NOUN)]","[ممنون دیجی, ممنون, کالا]","[[[0.045696888, -0.022505857, 0.045260634, -0....","[0.086360924, 0.023256088, 0.015749441, -0.081..."


In [21]:
all_data.to_csv("digiData.csv",encoding='utf-8-sig',index=False)

In [51]:
# Apply the function and create a temporary DataFrame
temp_sim_df = all_data.apply(lambda row: vectorSimilarity(row['cand_vector'], row['text_vector']), axis=1)

# Assign the two columns from the temporary DataFrame to your original DataFrame
all_data['candidates_sim_text_norm'] = temp_sim_df.apply(lambda x: x[0])
all_data['candidate_sim_candidate_norm'] = temp_sim_df.apply(lambda x: x[1])
all_data

<ipython-input-50-442f93aa5d37>:79: RuntimeWarning: invalid value encountered in divide
  candidate_sim_candidate_norm = 0.5 + (
<ipython-input-50-442f93aa5d37>:79: RuntimeWarning: invalid value encountered in divide
  candidate_sim_candidate_norm = 0.5 + (
<ipython-input-50-442f93aa5d37>:71: RuntimeWarning: invalid value encountered in divide
  candidates_sim_text_norm = 0.5 + (
<ipython-input-50-442f93aa5d37>:75: RuntimeWarning: All-NaN slice encountered
  candidate_sim_candidate_norm = candidate_sim_candidate / np.nanmax(
<ipython-input-50-442f93aa5d37>:81: RuntimeWarning: Mean of empty slice
  - np.nanmean(candidate_sim_candidate_norm, axis=0)
/usr/local/lib/python3.10/dist-packages/numpy/lib/nanfunctions.py:1872: RuntimeWarning: Degrees of freedom <= 0 for slice.
  var = nanvar(a, axis=axis, dtype=dtype, out=out, ddof=ddof,
<ipython-input-50-442f93aa5d37>:79: RuntimeWarning: invalid value encountered in divide
  candidate_sim_candidate_norm = 0.5 + (
<ipython-input-50-442f93aa5d37

,comment,processed,normalized,words,stems,lemmas,pos_tags,candidate_tags,cand_vector,text_vector,candidates_sim_text_norm,candidate_sim_candidate_norm
0,بنظرم در بین پنیر خامه‌ای ها از همه بهتره و بر...,{'normalized': 'بنظرم در بین پنیر خامه‌ای‌ها ا...,بنظرم در بین پنیر خامه‌ای‌ها از همه بهتره و بر...,"[بنظرم, در, بین, پنیر, خامه‌ای‌ها, از, همه, به...","[بنظر, در, بین, پنیر, خامه‌ای, از, همه, بهتره,...","[بنظر, در, بین, پنیر, خامه‌ای‌ها, از, همه, بهت...","[(بنظرم, VERB), (در, ADP), (بین, NOUN,EZ), (پن...","[پنیر, کیفیت, بین, خامه‌ای‌ها, برندها, بین پنیر]","[[[-0.04970275, -0.051454887, 0.020210866, 0.0...","[0.09832931, 0.016934274, 0.19831714, 0.065371...","[[0.6315147], [0.5235835], [0.44719887], [-1.1...","[[nan, 1.0486567, 0.018301696, 1.6128638, 0.48..."
1,خوب بود,"{'normalized': 'خوب بود', 'words': ['خوب', 'بو...",خوب بود,"[خوب, بود]","[خوب, بود]","[خوب, بود#است]","[(خوب, ADJ), (بود, VERB)]",[],[[]],"[0.0010530046, -0.00035910666, -0.00092692504,...",[],[]
2,خوش طعم و با کیفیت,"{'normalized': 'خوش‌طعم و با کیفیت', 'words': ...",خوش‌طعم و با کیفیت,"[خوش‌طعم, و, با, کیفیت]","[خوش‌طع, و, با, کیف]","[خوش‌طعم, و, با, کیفیت]","[(خوش‌طعم, NOUN), (و, CCONJ), (با, ADP), (کیفی...","[خوش‌طعم, کیفیت]","[[[-0.028009595, -0.018868402, 0.026083963, 0....","[0.030181762, -0.025148587, 0.032836657, 0.075...","[[-0.5], [1.5000001]]","[[nan, nan], [nan, nan]]"
3,عالی,"{'normalized': 'عالی', 'words': ['عالی'], 'ste...",عالی,[عالی],[عال],[عالی],"[(عالی, ADJ)]",[],[[]],"[-0.0006072971, -0.0010746294, -0.00013960322,...",[],[]
4,این طعم رو دوست داریم,"{'normalized': 'این طعم رو دوست داریم', 'words...",این طعم رو دوست داریم,"[این, طعم, رو, دوست, داریم]","[این, طع, رو, دوس, دار]","[این, طعم, رو, دوست, داشت#دار]","[(این, DET), (طعم, NOUN), (رو, ADP), (دوست, NO...","[دوست, طعم]","[[[-0.055163667, -0.07642987, -0.017042464, 0....","[-0.054517377, -0.090128876, -0.0017683336, 0....","[[1.5], [-0.5]]","[[nan, nan], [nan, nan]]"
...,...,...,...,...,...,...,...,...,...,...,...,...
415,خوشمزه و عالیه,"{'normalized': 'خوشمزه و عالیه', 'words': ['خو...",خوشمزه و عالیه,"[خوشمزه, و, عالیه]","[خوشمزه, و, عالیه]","[خوشمزه, و, عالیه]","[(خوشمزه, ADJ), (و, CCONJ), (عالیه, ADJ)]",[],[[]],"[0.0013152182, -0.00048353066, -0.0013701833, ...",[],[]
416,خوبه خوشمزس. دوسش داشتیم.,"{'normalized': 'خوبه خوشمزس. دوسش داشتیم.', 'w...",خوبه خوشمزس. دوسش داشتیم.,"[خوبه, خوشمزس, ., دوسش, داشتیم, .]","[خوبه, خوشمزس, ., دوس, داشت, .]","[خوبه, خوشمزس, ., دوس, داشت#دار, .]","[(خوبه, VERB), (خوشمزس, NOUN), (., PUNCT), (دو...","[دوسش, خوشمزس]","[[[0.0003717778, -0.037761122, -0.011944341, -...","[0.030795809, -0.0028205423, 0.01159153, -0.01...","[[1.5], [-0.5]]","[[nan, nan], [nan, nan]]"
417,لطیف و خوش طعم,"{'normalized': 'لطیف و خوش‌طعم', 'words': ['لط...",لطیف و خوش‌طعم,"[لطیف, و, خوش‌طعم]","[لطیف, و, خوش‌طع]","[لطیف, و, خوش‌طعم]","[(لطیف, ADJ), (و, CCONJ), (خوش‌طعم, ADJ)]",[],[[]],"[0.001214448, -0.0012010353, 0.0016182135, -0....",[],[]
418,ممنون دیجی کالا,"{'normalized': 'ممنون دیجی کالا', 'words': ['م...",ممنون دیجی کالا,"[ممنون, دیجی, کالا]","[ممنون, دیج, کالا]","[ممنون, دیج, کالا]","[(ممنون, NOUN,EZ), (دیجی, ADJ,EZ), (کالا, NOUN)]","[ممنون دیجی, ممنون, کالا]","[[[0.045696888, -0.022505857, 0.045260634, -0....","[0.086360924, 0.023256088, 0.015749441, -0.081...","[[1.8728784], [-0.4803841], [0.10750592]]","[[nan, 1.5000004, -0.5], [1.5000001, nan, 1.5]..."


In [52]:
all_data.to_csv("digiData.csv",encoding='utf-8-sig',index=False)

In [57]:
all_data['keywords'] = all_data.apply(
    lambda row: embedRankExtraction(
        np.array(row['candidate_tags']),
        row['candidates_sim_text_norm'],
        row['candidate_sim_candidate_norm']
    ), axis=1
)

<ipython-input-56-f3e45341b442>:116: UserWarning: total number of keyword candidates is 6, which is lower than your request keyword_num
  warnings.warn(
<ipython-input-56-f3e45341b442>:116: UserWarning: total number of keyword candidates is 2, which is lower than your request keyword_num
  warnings.warn(
<ipython-input-56-f3e45341b442>:116: UserWarning: total number of keyword candidates is 1, which is lower than your request keyword_num
  warnings.warn(
<ipython-input-56-f3e45341b442>:116: UserWarning: total number of keyword candidates is 4, which is lower than your request keyword_num
  warnings.warn(
<ipython-input-56-f3e45341b442>:116: UserWarning: total number of keyword candidates is 3, which is lower than your request keyword_num
  warnings.warn(
<ipython-input-56-f3e45341b442>:116: UserWarning: total number of keyword candidates is 5, which is lower than your request keyword_num
  warnings.warn(
<ipython-input-56-f3e45341b442>:116: UserWarning: total number of keyword candidat

In [58]:
all_data

,comment,processed,normalized,words,stems,lemmas,pos_tags,candidate_tags,cand_vector,text_vector,candidates_sim_text_norm,candidate_sim_candidate_norm,keywords
0,بنظرم در بین پنیر خامه‌ای ها از همه بهتره و بر...,{'normalized': 'بنظرم در بین پنیر خامه‌ای‌ها ا...,بنظرم در بین پنیر خامه‌ای‌ها از همه بهتره و بر...,"[بنظرم, در, بین, پنیر, خامه‌ای‌ها, از, همه, به...","[بنظر, در, بین, پنیر, خامه‌ای, از, همه, بهتره,...","[بنظر, در, بین, پنیر, خامه‌ای‌ها, از, همه, بهت...","[(بنظرم, VERB), (در, ADP), (بین, NOUN,EZ), (پن...","[پنیر, کیفیت, بین, خامه‌ای‌ها, برندها, بین پنیر]","[[[-0.04970275, -0.051454887, 0.020210866, 0.0...","[0.09832931, 0.016934274, 0.19831714, 0.065371...","[[0.6315147], [0.5235835], [0.44719887], [-1.1...","[[nan, 1.0486567, 0.018301696, 1.6128638, 0.48...","[بین پنیر, کیفیت, پنیر, بین, برندها, خامه‌ای‌ها]"
1,خوب بود,"{'normalized': 'خوب بود', 'words': ['خوب', 'بو...",خوب بود,"[خوب, بود]","[خوب, بود]","[خوب, بود#است]","[(خوب, ADJ), (بود, VERB)]",[],[[]],"[0.0010530046, -0.00035910666, -0.00092692504,...",[],[],[]
2,خوش طعم و با کیفیت,"{'normalized': 'خوش‌طعم و با کیفیت', 'words': ...",خوش‌طعم و با کیفیت,"[خوش‌طعم, و, با, کیفیت]","[خوش‌طع, و, با, کیف]","[خوش‌طعم, و, با, کیفیت]","[(خوش‌طعم, NOUN), (و, CCONJ), (با, ADP), (کیفی...","[خوش‌طعم, کیفیت]","[[[-0.028009595, -0.018868402, 0.026083963, 0....","[0.030181762, -0.025148587, 0.032836657, 0.075...","[[-0.5], [1.5000001]]","[[nan, nan], [nan, nan]]","[کیفیت, خوش‌طعم]"
3,عالی,"{'normalized': 'عالی', 'words': ['عالی'], 'ste...",عالی,[عالی],[عال],[عالی],"[(عالی, ADJ)]",[],[[]],"[-0.0006072971, -0.0010746294, -0.00013960322,...",[],[],[]
4,این طعم رو دوست داریم,"{'normalized': 'این طعم رو دوست داریم', 'words...",این طعم رو دوست داریم,"[این, طعم, رو, دوست, داریم]","[این, طع, رو, دوس, دار]","[این, طعم, رو, دوست, داشت#دار]","[(این, DET), (طعم, NOUN), (رو, ADP), (دوست, NO...","[دوست, طعم]","[[[-0.055163667, -0.07642987, -0.017042464, 0....","[-0.054517377, -0.090128876, -0.0017683336, 0....","[[1.5], [-0.5]]","[[nan, nan], [nan, nan]]","[دوست, طعم]"
...,...,...,...,...,...,...,...,...,...,...,...,...,...
415,خوشمزه و عالیه,"{'normalized': 'خوشمزه و عالیه', 'words': ['خو...",خوشمزه و عالیه,"[خوشمزه, و, عالیه]","[خوشمزه, و, عالیه]","[خوشمزه, و, عالیه]","[(خوشمزه, ADJ), (و, CCONJ), (عالیه, ADJ)]",[],[[]],"[0.0013152182, -0.00048353066, -0.0013701833, ...",[],[],[]
416,خوبه خوشمزس. دوسش داشتیم.,"{'normalized': 'خوبه خوشمزس. دوسش داشتیم.', 'w...",خوبه خوشمزس. دوسش داشتیم.,"[خوبه, خوشمزس, ., دوسش, داشتیم, .]","[خوبه, خوشمزس, ., دوس, داشت, .]","[خوبه, خوشمزس, ., دوس, داشت#دار, .]","[(خوبه, VERB), (خوشمزس, NOUN), (., PUNCT), (دو...","[دوسش, خوشمزس]","[[[0.0003717778, -0.037761122, -0.011944341, -...","[0.030795809, -0.0028205423, 0.01159153, -0.01...","[[1.5], [-0.5]]","[[nan, nan], [nan, nan]]","[دوسش, خوشمزس]"
417,لطیف و خوش طعم,"{'normalized': 'لطیف و خوش‌طعم', 'words': ['لط...",لطیف و خوش‌طعم,"[لطیف, و, خوش‌طعم]","[لطیف, و, خوش‌طع]","[لطیف, و, خوش‌طعم]","[(لطیف, ADJ), (و, CCONJ), (خوش‌طعم, ADJ)]",[],[[]],"[0.001214448, -0.0012010353, 0.0016182135, -0....",[],[],[]
418,ممنون دیجی کالا,"{'normalized': 'ممنون دیجی کالا', 'words': ['م...",ممنون دیجی کالا,"[ممنون, دیجی, کالا]","[ممنون, دیج, کالا]","[ممنون, دیج, کالا]","[(ممنون, NOUN,EZ), (دیجی, ADJ,EZ), (کالا, NOUN)]","[ممنون دیجی, ممنون, کالا]","[[[0.045696888, -0.022505857, 0.045260634, -0....","[0.086360924, 0.023256088, 0.015749441, -0.081...","[[1.8728784], [-0.4803841], [0.10750592]]","[[nan, 1.5000004, -0.5], [1.5000001, nan, 1.5]...","[ممنون دیجی, کالا, ممنون]"


In [59]:
all_data.to_csv("digiData.csv",encoding='utf-8-sig',index=False)

In [10]:
ind=['comment_candidates' , "comment_candidate_vector" , "comment_text_vector","keyword"]
# Create indexes
for i in ind:
    if not es.indices.exists(index=i):
        es.indices.create(index=i)

C:\Users\ZETTA\anaconda3\lib\site-packages\urllib3\connectionpool.py:1045: InsecureRequestWarning: Unverified HTTPS request is being made to host 'localhost'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(
C:\Users\ZETTA\anaconda3\lib\site-packages\urllib3\connectionpool.py:1045: InsecureRequestWarning: Unverified HTTPS request is being made to host 'localhost'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(
C:\Users\ZETTA\anaconda3\lib\site-packages\urllib3\connectionpool.py:1045: InsecureRequestWarning: Unverified HTTPS request is being made to host 'localhost'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(
C:\Users\ZETTA\anaconda3\lib\site-packages\urllib3\connectionpool.py:1045: I

In [11]:
bulk_upload(ind[0], all_data[['comment', 'candidate_tags']].to_dict(orient='records'))
bulk_upload(ind[1], all_data[['comment', 'cand_vector']].to_dict(orient='records'))
bulk_upload(ind[2], all_data[['comment', 'text_vector']].to_dict(orient='records'))
bulk_upload(ind[3], all_data[['comment', 'keywords']].to_dict(orient='records'))

C:\Users\ZETTA\anaconda3\lib\site-packages\urllib3\connectionpool.py:1045: InsecureRequestWarning: Unverified HTTPS request is being made to host 'localhost'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(
C:\Users\ZETTA\anaconda3\lib\site-packages\urllib3\connectionpool.py:1045: InsecureRequestWarning: Unverified HTTPS request is being made to host 'localhost'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(
C:\Users\ZETTA\anaconda3\lib\site-packages\urllib3\connectionpool.py:1045: InsecureRequestWarning: Unverified HTTPS request is being made to host 'localhost'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(
C:\Users\ZETTA\anaconda3\lib\site-packages\urllib3\connectionpool.py:1045: I

In [4]:
all_data = pd.read_csv("digiData_final.csv")